In [1]:
import pandas as pd
from math import sqrt, ceil
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import BaggingClassifier 

## 1. Monte um passo a passo para o algoritmo Random Forest

**1. Amostragem de Dados:**
- Para cada árvore de decisão no conjunto, é feita uma amostragem aleatória dos dados de treinamento.
- Essa amostragem é feita com substituição, o que significa que cada árvore pode ter exemplos repetidos ou não ter alguns exemplos.

**2. Construção das Árvores:**

- Para cada árvore de decisão, é feita a construção da árvore usando a amostra de dados correspondente.
- Durante a construção da árvore, em cada nó, o algoritmo seleciona um subconjunto aleatório de características (variáveis) para determinar a melhor divisão.
- Essa aleatoriedade na seleção de características ajuda a reduzir a correlação entre as árvores e torna o modelo mais robusto.

**3. Votação por Maioria:**

- Após a construção de todas as árvores, o Random Forest faz previsões combinando as previsões de cada árvore.
- No caso de classificação, a previsão final é determinada por votação por maioria. Cada árvore tem um voto igual e a classe com mais votos é selecionada como a previsão final.
- No caso de regressão, a previsão final é a média das previsões de todas as árvores.

## 2. Explique com suas palavras o Random Forest

Random Forest é um algoritmo que utiliza um conjunto de árvores de decisão como estimadores base. No Random Forest, cada árvore de decisão é construída de forma aleatória, tornando o algoritmo mais robusto e geralmente mais preciso porque a combinação de várias árvores de decisão independentes e a aleatoriedade na seleção de características ajudam a reduzir a variância do modelo e evitar overfitting.

## 3. Qual a diferença entre Bagging e Random Forest?

A principal diferença entre eles seria a de que o Bagging cria um conjunto de estimadores ou classificadores base usando amostras de treinamento aleatórias com substituição. Os estimadores ou classificadores nesse caso poderiam ser qualquer algoritmo.

Já o Random Forest utiliza árvores de decisão como estimadores base. Ele cria um conjunto de árvores de decisão, onde cada árvore é treinada de forma independente em uma amostra de treinamento aleatória com substituição.

## 4. Implementação em Python do Bootstrap, Feature Selection e RandomForest

Será utilizada a base de dados 'credit_scoring' disponibilizada no material da aula. Como o intuito é exemplificar a implementação do algoritmo, não serão utilizadas todas as variáveis disponíveis nessa base de dados. 

In [2]:
df = (
    pd.read_csv('credit_scoring.csv')
    .dropna()
    .drop(columns=['data_ref', 'id_cliente', 'tipo_renda', 'educacao'])
    .drop(columns=['tipo_residencia', 'renda', 'estado_civil'])
    .astype({'tempo_emprego': 'int32', 'qt_pessoas_residencia': 'int32'})
    .replace({'posse_de_veiculo': {True: 1, False: 0}})
    .replace({'posse_de_imovel': {True: 1, False: 0}})
    .replace({'sexo': {'F': 1, 'M': 0}})
)

Abaixo temos um exemplo da base de dados que será utilizada.

In [201]:
df.head()

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,idade,tempo_emprego,qt_pessoas_residencia,mau
0,1,1,1,0,49,8,2,0
1,0,1,0,0,60,6,2,0
2,1,1,0,0,28,0,2,0
3,1,0,1,0,60,1,2,0
4,1,0,0,0,47,8,2,0


**Bootstrap e Feature Selection** \
A função abaixo será criada para construir um novo DataFrame (baseado no original) usando a função sample do pandas para implementar o Bootstrap e Feature Selection de forma randômica.

In [103]:
def bootstrap_fs(data: pd.DataFrame) -> pd.DataFrame:
    # a amostra será criada com 80% da quantidade total de linhas do DataFrame original
    linhas = int(data.shape[0] * .80)
    
    # valor de p, quantidade total de colunas
    p = data.shape[1]
    
    # valor de m, raiz quadrada de p
    m = ceil(sqrt(p))
    
    # cria um DataFrame com bootstrap e substituições
    dt = data.sample(n = linhas, replace=True)
    
    # guarda a variável resposta
    y = dt['mau']
    
    # remove da amostra a variável resposta
    dt.drop(columns=['mau'], inplace=True)
    
    # aplica o Feature Selection
    dt = dt.sample(n = m, axis=1)
    
    # como o Feature Selection é randômico, 
    # ordena alfabeticamente as colunas da amostra para manter um padrão
    dt = dt.reindex(sorted(dt.columns), axis=1)
    
    # devolve a variável resposta para a amostra
    dt['mau'] = y
    
    return dt

Como as colunas das amostras podem variar, devido ao Feature Selection, a função abaixo cria um DataFrame com os dados que serão utilizados para fazer a previsão apenas com as variáveis que estão na amostra. Funciona como uma interseção entre os dados para previsão e as variáveis disponíveis na amostra

In [5]:
def criar_dataframe_previsao(amostra: pd.DataFrame, dados: dict) -> pd.DataFrame:
    # dados para previsão referente as variáveis disponíveis na amostra
    valores = {}
    
    # percorre os nomes das variáveis referente a interseção das colunas da amostra 
    # e as chaves dos dados para previsão
    for nome_variavel in set(amostra.columns).intersection(dados.keys()):
        valores[nome_variavel] = [dados[nome_variavel]]
        
    dt = pd.DataFrame(valores)
    
    # ordena alfabeticamente as colunas da amostra para manter um padrão
    dt = dt.reindex(sorted(dt.columns), axis=1)
    
    return dt

A função abaixo recebe por parâmetro uma amostra e os dados para previsão. Essa função será aplicada para cada amostra e iremos armazenar a previsão em cada uma delas para realizarmos a agregação posteriormente.

In [114]:
def realizar_previsao(amostra: pd.DataFrame, dados: dict):
    X = amostra.drop(columns=['mau'])
    y = amostra['mau']
    
    dtree = DecisionTreeClassifier(random_state = 1)
    dtree.fit(X, y)
    
    dados_para_previsao = criar_dataframe_previsao(amostra, dados)
    
    return dtree.predict(dados_para_previsao)[0]

Abaixo criamos uma lista com 10 amostras criadas com Bootstrap e Feature Selection

In [123]:
amostras = [bootstrap_fs(df) for _ in range(0, 10)]

Abaixo é definido um dicionário com os dados que serão usados para a previsão

In [181]:
dados = {
    'sexo': 0, # homen
    'posse_de_veiculo': 1,
    'posse_de_imovel': 0,
    'qtd_filhos': 4,
    'idade': 42,
    'tempo_emprego': 1, # 1 ano
    'qt_pessoas_residencia': 6
}

Abaixo criamos um DataFrame com o resultado da previsão de cada amostra

In [186]:
previsoes = pd.DataFrame({'mau': [
    realizar_previsao(amostra, dados) for amostra in amostras
]})

In [187]:
previsoes

,mau
0,0
1,1
2,0
3,0
4,0
5,0
6,0
7,1
8,0
9,0


In [202]:
# Agregação dos resultados dos modelos
freq = pd.DataFrame(previsoes.value_counts(), columns=['frequencia'])
freq

,frequencia
mau,
0,8
1,2


Nesse exemplo de 10 amostras criadas usadno Bootstrap e Feature Selection, usando o modelo DecisionTreeClassifier, **8** modelos apontaram que os dados usados para previsão se referem a um indivíduo considerado bom pagador e **2** modelos um mau pagador.

### BaggingClassifier do Scikit-Learn

O processo demonstrado anteriormente serviu apenas para exemplificar os passos para construção do RandomForest com Bagging, Bootstrap e Feature Selection. 

Na biblioteca Scikit-Learn existe um algorítimo pronto chamado BaggingClassifier. Com ele podemos rodar todos esses passos de uma só vez implementando apenas construção de um só modelo.

Como o BaggingClassifier podemos usar qualquer estimador, sendo que por padrão ele implementa o DecisionTreeClassifier.

Também podemos definir a quantidade e estimadores que desejamos utilizar, a quantidade de amostras usando ou não bootstrap com substituições e a quantidade de variáveis (features).

Veja um exemplo da implementação do BaggingClassifier abaixo:

In [10]:
# separando as variáveis preditivas e de resposta
X = df.drop(columns=['mau'])
X = X.reindex(sorted(X.columns), axis=1)

y = df['mau']

In [16]:
# criando um DataFrame com os dados que serão usados para previsão
dados_para_previsao = criar_dataframe_previsao(df, {
    'sexo': 1, # mulher
    'posse_de_veiculo': 1,
    'posse_de_imovel': 1,
    'qtd_filhos': 1,
    'idade': 29,
    'tempo_emprego': 4,
    'qt_pessoas_residencia': 3
})

In [17]:
# construindo o BaggingClassifier
bag = BaggingClassifier(
    random_state = 1, 
    
    # Definindo o número de estimadores. Por padrão usa o DecisionTreeClassifier
    n_estimators = 10, 
    
    # quantidade de amostras para treinar cada estimador
    max_samples = 10, 
    
    # quantidade de variáveis para treinar cada estimador
    max_features = 3,
)

# treina o modelo
bag.fit(X, y)

# requisita a previsão
resultado = bag.predict(dados_para_previsao)[0]

print(f'É mau pagador? (0 = Não 1 = Sim): {resultado}')

É mau pagador? (0 = Não 1 = Sim): 0
